# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])

import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from nltk.corpus import stopwords
from sklearn.utils.multiclass import type_of_target
import sklearn
from sklearn.model_selection import GridSearchCV


[nltk_data] Downloading package punkt to /home/edgart/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/edgart/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/edgart/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 0.22.2.post1.


In [4]:
categories = ['related', 'request',
       'offer', 'aid_related', 'medical_help', 'medical_products',
       'search_and_rescue', 'security', 'military', 'child_alone',
       'water', 'food', 'shelter', 'clothing', 'money', 'missing_people',
       'refugees', 'death', 'other_aid', 'infrastructure_related',
       'transport', 'buildings', 'electricity', 'tools', 'hospitals',
       'shops', 'aid_centers', 'other_infrastructure', 'weather_related',
       'floods', 'storm', 'fire', 'earthquake', 'cold', 'other_weather',
       'direct_report']

In [5]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql("SELECT * FROM responses", engine)
X = df["message"].values
Y = df.iloc[:, 4:].values
categories = df.iloc[:, 4:].columns

In [25]:
categories = df.iloc[:, 4:].columns.values
categories

array(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers',
       'other_infrastructure', 'weather_related', 'floods', 'storm',
       'fire', 'earthquake', 'cold', 'other_weather', 'direct_report'],
      dtype=object)

In [5]:
X.shape

(9631,)

In [6]:
Y.shape

(9631, 36)

In [7]:
Y

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 1.]])

In [6]:
X

array(['Weather update - a cold front from Cuba that could pass over Haiti',
       'Is the Hurricane over or is it not over',
       'Looking for someone but no name', ...,
       'shikarpur: Volunteers needed, food and medical help needed',
       'Dadu: Needs water, food, medicines. boats',
       'Umer kot: Sindh: Shelter and Camp needed as people are stranded due to flood:'],
      dtype=object)

In [9]:
df.iloc[0]

id                                                                        2
message                   Weather update - a cold front from Cuba that c...
original                  Un front froid se retrouve sur Cuba ce matin. ...
genre                                                                direct
related                                                                   1
request                                                                   0
offer                                                                     0
aid_related                                                               0
medical_help                                                              0
medical_products                                                          0
search_and_rescue                                                         0
security                                                                  0
military                                                                  0
child_alone 

In [7]:
df.iloc[0]['message']

'Weather update - a cold front from Cuba that could pass over Haiti'

In [8]:
df.iloc[0]['original']

'Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi'

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    stop_words = stopwords.words("english")

    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]


    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(KNeighborsClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

In [12]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x7fe070844f28>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',

In [13]:
pipeline.score

<function sklearn.pipeline.Pipeline.score(self, X, y=None, sample_weight=None)>

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
i = 0 
for name in categories:
    true = np.take(y_test, i, axis = 1)
    pred = np.take(y_pred, i, axis = 1)
    print('---------')
    print(name)
    print(classification_report(true, pred))
    i += 1

---------
related
              precision    recall  f1-score   support

         0.0       0.39      0.16      0.23       822
         1.0       0.66      0.87      0.75      1558
         2.0       0.00      0.00      0.00        28

    accuracy                           0.62      2408
   macro avg       0.35      0.34      0.33      2408
weighted avg       0.56      0.62      0.56      2408

---------
request
              precision    recall  f1-score   support

         0.0       0.66      0.87      0.75      1561
         1.0       0.39      0.16      0.23       847

    accuracy                           0.62      2408
   macro avg       0.52      0.51      0.49      2408
weighted avg       0.56      0.62      0.56      2408

---------
offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2407
         1.0       0.00      0.00      0.00         1

    accuracy                           1.00      2408
   macro avg       0.5

/mnt/d/data-scientist/dev/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98      2314
         1.0       0.00      0.00      0.00        94

    accuracy                           0.96      2408
   macro avg       0.48      0.50      0.49      2408
weighted avg       0.92      0.96      0.94      2408

---------
electricity
              precision    recall  f1-score   support

         0.0       0.99      1.00      1.00      2386
         1.0       0.00      0.00      0.00        22

    accuracy                           0.99      2408
   macro avg       0.50      0.50      0.50      2408
weighted avg       0.98      0.99      0.99      2408

---------
tools
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2403
         1.0       0.00      0.00      0.00         5

    accuracy                           1.00      2408
   macro avg       0.50      0.50      0.50      2408
weighted avg       1.00      1.00   

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
parameters = {
    'clf__estimator__n_neighbors': [3,5,11,15],
    'clf__estimator__weights': ['uniform', 'distance']
}

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        p

In [18]:
y_pred_cv =cv.predict(X_test)


In [19]:
i = 0 
for name in categories:
    true = np.take(y_test, i, axis = 1)
    pred = np.take(y_pred_cv, i, axis = 1)
    print('---------')
    print(name)
    print(classification_report(true, pred))
    i += 1

---------
related
              precision    recall  f1-score   support

         0.0       0.39      0.12      0.18       822
         1.0       0.65      0.91      0.76      1558
         2.0       0.00      0.00      0.00        28

    accuracy                           0.63      2408
   macro avg       0.35      0.34      0.31      2408
weighted avg       0.56      0.63      0.55      2408

---------
request
              precision    recall  f1-score   support

         0.0       0.65      0.91      0.76      1561
         1.0       0.37      0.09      0.15       847

    accuracy                           0.62      2408
   macro avg       0.51      0.50      0.45      2408
weighted avg       0.55      0.62      0.54      2408

---------
offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      2407
         1.0       0.00      0.00      0.00         1

    accuracy                           1.00      2408
   macro avg       0.5

/mnt/d/data-scientist/dev/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99      2374
         1.0       0.00      0.00      0.00        34

    accuracy                           0.99      2408
   macro avg       0.49      0.50      0.50      2408
weighted avg       0.97      0.99      0.98      2408

---------
weather_related
              precision    recall  f1-score   support

         0.0       0.85      0.99      0.92      2054
         1.0       0.21      0.01      0.02       354

    accuracy                           0.85      2408
   macro avg       0.53      0.50      0.47      2408
weighted avg       0.76      0.85      0.79      2408

---------
floods
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      2353
         1.0       0.00      0.00      0.00        55

    accuracy                           0.98      2408
   macro avg       0.49      0.50      0.49      2408
weighted avg       0.95      0.

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [20]:
cv.best_score_

0.16862674807579867

### 9. Export your model as a pickle file

In [21]:
import pickle
filename = 'finalized_model.pkl'
pickle.dump(cv, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [22]:
print(cv.best_params_)

{'clf__estimator__n_neighbors': 15, 'clf__estimator__weights': 'uniform'}


In [26]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
